In [61]:
from  urllib.request import urlopen
import json
import nibabel as nib

url = "http://neurovault.org/api/atlases/"
response = urlopen(url)
data = response.read().decode("utf-8")
atlases = json.loads(data)

In [51]:
for result in atlases['results']:
    print(result['name'])

JHU ICBM labels 2mm
JHU ICBM labels 1mm
HarvardOxford sub maxprob thr25 2mm
HarvardOxford cort maxprob thr25 2mm
HarvardOxford cort maxprob thr25 1mm
HarvardOxford sub maxprob thr25 1mm
JHU ICBM tracts maxprob thr25 2mm
JHU ICBM tracts maxprob thr25 1mm
MNI maxprob thr25 1mm
MNI maxprob thr25 2mm
Juelich maxprob thr25 1mm
Juelich maxprob thr25 2mm
Thalamus maxprob thr25 2mm
Thalamus maxprob thr25 1mm
Talairach labels 2mm
Talairach labels 1mm
AAL2
AAL


In [69]:
hocort_2mm_idx = next(index for (index, d) in enumerate(atlases["results"]) if d["name"] == "HarvardOxford cort maxprob thr25 2mm")
atlasurl = atlases['results'][hocort_2mm_idx]['file']
atlasurl

'http://neurovault.org/media/images/262/HarvardOxford-cort-maxprob-thr25-2mm.nii.gz'

In [80]:
import os
import numpy as np
atlas_fname = os.path.abspath('/home/jflournoy/code/arbitrary_nii_extraction/HarvardOxford-cort-maxprob-thr25-2mm.nii.gz')

atlas = nib.load(atlas_fname)

import matplotlib.pyplot as plt

def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
        
atlas_data = atlas.get_data()

In [88]:
fig = plt.plot(atlas_data[26, :, :])


In [ ]:
mask_file =
data_file = 
timeseries = 


    #given a list of mask files and data files (copes, preprocessed data, etc)
    #returns a data frame of values extracted from those masks
    #the column name is taken based on the mask file in the first row (with file extension stripped)
   
    if len(mask_files) != len(data_files):
        print("mask and data files don't match!")
        
    maskname = mask_files[0].split('/')
    maskname = maskname[len(maskname)-1].strip().split('.')[0]
    
    if timeseries:
        tsdata = []
    else:
        tsdata = pd.Series(data=np.empty(len(mask_files)))
        tsdata[:] = np.NAN

    
    for i,m in enumerate(mask_files):
        ts_out_file = tempfile.NamedTemporaryFile()
        #nipype wrapper for fslmeants
        meants = fsl.ImageMeants()
        meants.inputs.in_file = data_files[i]
        meants.inputs.mask = os.path.abspath(os.path.expanduser(m))
        meants.inputs.out_file = ts_out_file.name
        result = meants.run()
        
        f = open(result.outputs.out_file,'r')
        temp = [line.strip() for line in f.readlines()] #remove the newline character \n
        
        if timeseries:
            tsdata.append(temp)
            print('finished file %s' %data_files[i])
        else:
            tsdata[i] = temp[0]
            
        f.close()
    
    if timeseries:
        output = tsdata
    else:
        output = pd.DataFrame(data=tsdata,columns=[maskname])
    
    return(output)    